The problem is predicting updrs_1, updrs_2, updrs_3 and updrs_4 values at a given visit_month and potential 6, 12 and 24 month visits after the current visit_month.

Build models using linear regression, svm, logistic regression, random forests

for lr, group 4 percentiles

In [1]:
import pandas as pd
import numpy as np

train_clinical = '~/School/EE445/kaggle/train_clinical_data.csv'
train_peptides = '~/School/EE445/kaggle/train_peptides.csv'
train_supp_clinical = '~/School/EE445/kaggle/supplemental_clinical_data.csv'
train_proteins = '~/School/EE445/kaggle/train_proteins.csv'

In [2]:
# load data
clinical_data = pd.read_csv(train_clinical)
protein_data = pd.read_csv(train_proteins)
peptide_data = pd.read_csv(train_peptides)
supp_data = pd.read_csv(train_supp_clinical)

### Initialize DF:

In [3]:
def Get_ALL_Protein_Names(df_protein):
    return df_protein['UniProt'].unique().tolist()

def Get_Protein_Names(df_protein, visit_id):
    return protein_data.loc[protein_data['visit_id'] == visit_id]['UniProt'].tolist()

def Get_Patient_Info(df_clinical):
    return df_clinical.columns[0:3].tolist() + [df_clinical.columns[-1]]

def Get_Targets(df_clinical):
    return df_clinical.columns[3:7].tolist()

def Get_Protein_Visit_ID(df_protein):
    return df_protein['visit_id'].unique().tolist()

def Get_Clinical_Visit_ID(df_clinical):
    return df_clinical['visit_id'].tolist()

def Get_NPX(df_protein, visit_id):
    return df_protein.loc[df_protein['visit_id'] == visit_id]['NPX'].tolist()

def Create_Patient_Data_Dict(visit_id):
    pat_id, vis_mon = visit_id.split('_')
    return {'patient_id': int(pat_id), 'visit_month': int(vis_mon), 'visit_id': visit_id }

def Create_Protein_Data_Dict(df_protein, NPX_data, visit_id):
    return dict(zip(Get_Protein_Names(df_protein, visit_id), NPX_data)) | Create_Patient_Data_Dict(visit_id)
    
def Find_Visit_ID_Only_Protein_Measured(df_clinical, df_protein):
    protein_list = Get_Protein_Visit_ID(df_protein)
    clinical_list = Get_Clinical_Visit_ID(df_clinical)
    only_protein = [visit_id for visit_id in protein_list if visit_id not in clinical_list]
    return only_protein

def Find_Visit_ID_Only_UPDRS_Measured(df_clinical, df_protein):
    protein_list = Get_Protein_Visit_ID(df_protein)
    clinical_list = Get_Clinical_Visit_ID(df_clinical)
    only_UPDRS = [visit_id for visit_id in clinical_list if visit_id not in protein_list]
    return only_UPDRS

def Find_Visit_ID_Protein_and_UPDRS_Measured(df_clinical, df_protein):
    return [visit_id for visit_id in Get_Clinical_Visit_ID(df_clinical) if visit_id in Get_Protein_Visit_ID(df_protein)]

def Add_Rows_Only_Protein_Measured(df, only_prot_visit_id, df_protein):
    for visit_id in only_prot_visit_id:
        NPX_data = Get_NPX(df_protein, visit_id)
        protein_info = pd.DataFrame(Create_Protein_Data_Dict(df_protein, NPX_data, visit_id), index=[0])
        df = pd.concat([df, protein_info], axis = 0)
    return df

def Add_Protein_Data(df, df_clinical, df_protein):
    prot_visit_id = Find_Visit_ID_Protein_and_UPDRS_Measured(df_clinical, df_protein)
    for visit_id in prot_visit_id:
        NPX_data = Get_NPX(df_protein, visit_id)
        #protein_info = pd.DataFrame(Create_Protein_Data_Dict(df_protein, NPX_data, visit_id), index=[0])
        df.loc[df.index[df['visit_id']==visit_id], Get_Protein_Names(df_protein, visit_id)] = NPX_data
    return df

def Sort_and_Index(df):
    df = df.sort_values(by=['patient_id', 'visit_month'])
    df = df.reset_index()
    return df.drop(['index'], axis = 1)

def Create_Combined_DataFrame(df_clinical, df_protein):
    protein_cols = Get_ALL_Protein_Names(df_protein)
    patient_cols = Get_Patient_Info(df_clinical)
    target_cols = Get_Targets(df_clinical)
    df_with_UniProt_cols = pd.DataFrame(columns = protein_cols, index=df_clinical.index)
    clinical_left = df_clinical[patient_cols]
    clinical_right = df_clinical[target_cols]
    df = pd.concat([clinical_left, df_with_UniProt_cols, clinical_right], axis = 1)
    only_p = Find_Visit_ID_Only_Protein_Measured(clinical_data, protein_data)
    df = Add_Rows_Only_Protein_Measured(df, only_p, protein_data)
    df = Sort_and_Index(df)
    return Add_Protein_Data(df, df_clinical, df_protein)

In [4]:
df = Create_Combined_DataFrame(clinical_data, protein_data)

In [5]:
df

,visit_id,patient_id,visit_month,upd23b_clinical_state_on_medication,O00391,O00533,O00584,O14498,O14773,O14791,...,P32754,P60174,Q13449,Q99683,Q99829,Q9UKV8,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,55,0,NaN,11254.3,732430.0,39585.8,41526.9,31238.0,4202.71,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6.0,15.0,NaN
1,55_3,55,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,25.0,NaN
2,55_6,55,6,NaN,13163.6,630465.0,35220.8,41295.0,26219.9,4416.42,...,NaN,NaN,49927.5,121417.0,NaN,NaN,8.0,10.0,34.0,NaN
3,55_9,55,9,On,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,9.0,30.0,0.0
4,55_12,55,12,On,15257.6,815083.0,41650.9,39763.3,30703.6,4343.60,...,NaN,20088.2,45519.2,121322.0,25589.4,65762.6,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,65043_48,65043,48,Off,10589.6,902434.0,44890.8,38771.5,20188.3,3137.07,...,57323.9,NaN,NaN,96827.3,14019.1,39046.7,7.0,6.0,13.0,0.0
2656,65043_54,65043,54,Off,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,8.0,11.0,1.0
2657,65043_60,65043,60,Off,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6.0,16.0,1.0
2658,65043_72,65043,72,Off,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,14.0,1.0


In [6]:
df.describe()

,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,P32754,P60174,Q13449,Q99683,Q99829,Q9UKV8,updrs_1,updrs_2,updrs_3,updrs_4
count,2660.000000,2660.000000,764.000000,1.112000e+03,1100.000000,1032.000000,1047.000000,942.000000,1113.000000,1050.000000,...,778.000000,729.000000,909.000000,933.000000,489.000000,907.000000,2614.000000,2613.00000,2590.000000,1577.000000
mean,32625.830827,30.839098,11641.264435,5.111649e+05,26505.529157,27305.934884,17688.295406,3004.990691,126151.780054,50773.474638,...,73994.153085,15290.254403,28216.340372,102498.454662,15393.157280,69924.939658,7.110559,6.74359,19.421236,1.861763
std,18538.098639,25.226370,2817.003530,2.357357e+05,10705.152540,8446.187506,7166.325369,1142.159575,72748.393517,21382.028764,...,31142.479712,5378.119619,13279.201144,21270.573763,5805.729783,25943.573085,5.525955,6.32323,15.000289,3.022112
min,55.000000,0.000000,873.778000,5.971820e+04,591.103000,8945.340000,2811.120000,336.517000,10717.400000,5806.840000,...,12718.200000,2710.320000,223.218000,47187.900000,3364.640000,3439.810000,0.000000,0.00000,0.000000,0.000000
25%,16572.000000,9.000000,9736.857500,3.490590e+05,19941.075000,21123.650000,12920.050000,2189.087500,70560.600000,37008.975000,...,53235.800000,11700.900000,18613.500000,88157.000000,11489.900000,54132.600000,3.000000,1.00000,6.000000,0.000000
50%,29313.000000,24.000000,11546.400000,4.834425e+05,26529.700000,26624.000000,17399.600000,2865.460000,116900.000000,50375.800000,...,68369.500000,14906.200000,25872.100000,100530.000000,14420.100000,67571.700000,6.000000,5.00000,19.000000,0.000000
75%,50149.000000,48.000000,13383.025000,6.485572e+05,33222.800000,32459.275000,22077.050000,3593.147500,164947.000000,63446.700000,...,88072.825000,18229.000000,35692.200000,114190.000000,18626.900000,83879.300000,10.000000,10.00000,29.000000,3.000000
max,65043.000000,108.000000,21361.800000,1.806980e+06,66252.400000,65347.900000,49695.600000,9352.640000,538862.000000,137369.000000,...,202714.000000,33801.800000,94180.500000,196434.000000,37061.700000,226139.000000,33.000000,40.00000,86.000000,20.000000


### Validating The DF:

In [7]:
# Testing n random UniProt and NPX from protein_data match DF
n = 1000
results = []
for i in range(n):
    sample = protein_data.sample()
    p_id = sample['patient_id'].values[0]
    vm = sample['visit_month'].values[0]
    up = sample['UniProt'].values[0]
    npx = sample['NPX'].values[0]
    #print('Check #:',i)
    #print('Patient ID:', p_id)
    #print('Visit Month:', vm)
    #print('UniProt:', up)
    #print('NPX from protein_data:', npx)
    result = float(df[up].loc[(df['patient_id'] == p_id)&(df['visit_month'] == vm)].values)
    #print('NPX from df:', result)
    #print('Pass?', result == npx)
    #print('\n')
    if result == npx:
        results.append(True)
print('Percentage Correct:', sum(results)/n)
    

Percentage Correct: 1.0


In [8]:
protein_cols = Get_ALL_Protein_Names(protein_data)
target_cols = Get_Targets(clinical_data)

# Testing n random samples from DF match protein_data
n = 1000
results = []
for i in range(n):
    sample = df.sample()
    valid_protein_columns = sample[protein_cols].dropna(axis=1)
    temp_df = pd.concat([pd.DataFrame(valid_protein_columns.columns, columns=['UniProt']),
                         pd.DataFrame(valid_protein_columns.values.transpose(), columns=['NPX'])], axis=1)
    prot_df = protein_data[['UniProt', 'NPX']].loc[protein_data['visit_id'] == sample['visit_id'].values[0]]
    prot_df = prot_df.sort_values('UniProt')
    temp_df = temp_df.sort_values('UniProt')
    prot_df = prot_df.reset_index(drop=True)
    temp_df = temp_df.reset_index(drop=True)
    truth_df = temp_df.sort_values('UniProt') == prot_df.sort_values('UniProt')
    if truth_df.sum().sum() == truth_df.shape[0]*2:
        results.append(True)
print('percentage correct:', sum(results)/n)

percentage correct: 1.0


In [9]:
protein_cols = Get_ALL_Protein_Names(protein_data)
target_cols = Get_Targets(clinical_data)

# Testing the rows where only protein measurements were taken
test_visit_ID = Find_Visit_ID_Only_Protein_Measured(clinical_data, protein_data)

# Loop over visit_id where only protein measurements were taken
for vis_id in test_visit_ID:
    # Retrieve Protein and NPX values from df, drop all NAN columns
    row_df = df[protein_cols].loc[df['visit_id']==vis_id]
    row_df = row_df[sorted(protein_cols)].dropna(axis=1)
    # Retrieve Protein and NPX
    prodat_df = protein_data.loc[protein_data['visit_id']==vis_id][['UniProt', 'NPX']]
    # Check that protein ids match
    print('Visit ID:',vis_id)
    print('Protein_IDs match:', row_df.columns.tolist() == prodat_df['UniProt'].tolist())
    # Check that NPX values match
    print('NPX match:', row_df.values.tolist()[0] == prodat_df['NPX'].tolist())
    # Check that the UPDRs are all NaN
    print('UPDRS are NaN:', df[target_cols].loc[df['visit_id']==vis_id].isna().sum().sum()==4) 
    print('\n')

Visit ID: 23175_3
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 57468_3
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 2660_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 5027_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 5036_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 5178_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 7117_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 7151_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 11686_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 12636_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 13968_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 14035_6
Protein_IDs match: True
NPX match: True
UPDRS are NaN: True


Visit ID: 14450_6
Protein_IDs match: True
NPX 

### Create training data (all visit_months == 0 OR 3)

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [106]:
target_cols = Get_Targets(clinical_data)
protein_cols = Get_ALL_Protein_Names(protein_data)

# train the model using measurements taken from months 0 and 3
X_train = df.drop(target_cols + ['visit_id', 'patient_id'], axis=1).loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = df[target_cols].loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = y_train.drop('updrs_4', axis=1)

# drop NaN rows
# Get indices with NaN in row
inds = pd.isnull(y_train).any(1).to_numpy().nonzero()[0].tolist()
inds = y_train.index[inds].tolist()
X_train = X_train.drop(inds, axis=0)
y_train = y_train.dropna()

In [107]:
y_train.isna().sum()

updrs_1    0
updrs_2    0
updrs_3    0
dtype: int64

In [108]:
#check that indices between X_train and y_train match
print((X_train.index == y_train.index).sum() == len(X_train.index))

True


### Create preprocessing pipelines

In [109]:
# Define transforms on numeric types
numeric_transformer_X = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

numeric_transformer_y = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Define transforms on categorical types
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Construct ColumnTransformer object for X
preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_X, protein_cols + ['visit_month']),
        ('cat', categorical_transformer, [Get_Patient_Info(clinical_data)[-1]])
    ],
    remainder='passthrough',
    verbose=True,
    verbose_feature_names_out=True)

## 25th Percentile Training

In [110]:
# Create columns with boolean labels that depend on whether the entry > the 25th percentile of the column
for col in y_train.columns:
    percentile_25 = y_train[col].quantile(0.25)
    y_train.loc[:, col+'_bool'] = y_train[col] > percentile_25
    
# throw out numeric columns
y_train_bool_cols = y_train.columns[3:6].tolist()
y_train = y_train[y_train_bool_cols]

In [111]:
# Train-Test-Split Method
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

### UPDRS_1

In [112]:
# Train the model to predict updrs_1_bool
clf1_25 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_1_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [113]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf1_25, preprocessor_X.fit_transform(X_train), y_train['updrs_1_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.65753425, 0.60273973, 0.52777778, 0.52777778, 0.61111111])

### UPDRS_2

In [114]:
# Train the model to predict updrs_1_bool
clf2_25 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_2_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [115]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf2_25, preprocessor_X.fit_transform(X_train), y_train['updrs_2_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.69863014, 0.67123288, 0.68055556, 0.70833333, 0.68055556])

### UPDRS_3

In [116]:
# Train the model to predict updrs_1_bool
clf3_25 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_3_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [117]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf3_25, preprocessor_X.fit_transform(X_train), y_train['updrs_3_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.7260274 , 0.75342466, 0.72222222, 0.73611111, 0.69444444])

## 50th Percentile Training

In [118]:
target_cols = Get_Targets(clinical_data)
protein_cols = Get_ALL_Protein_Names(protein_data)

# train the model using measurements taken from months 0 and 3
X_train = df.drop(target_cols + ['visit_id', 'patient_id'], axis=1).loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = df[target_cols].loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = y_train.drop('updrs_4', axis=1)

# drop NaN rows
# Get indices with NaN in row
inds = pd.isnull(y_train).any(1).to_numpy().nonzero()[0].tolist()
inds = y_train.index[inds].tolist()
X_train = X_train.drop(inds, axis=0)
y_train = y_train.dropna()

In [119]:
# Create columns with boolean labels that depend on whether the entry > the 25th percentile of the column
for col in y_train.columns:
    percentile_50 = y_train[col].quantile(0.50)
    y_train.loc[:, col+'_bool'] = y_train[col] > percentile_50
    
# throw out numeric columns
y_train_bool_cols = y_train.columns[3:6].tolist()
y_train = y_train[y_train_bool_cols]

### UPDRS_1

In [120]:
# Train the model to predict updrs_1_bool
clf1_50 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_1_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [121]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf1_50, preprocessor_X.fit_transform(X_train), y_train['updrs_1_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.50684932, 0.54794521, 0.59722222, 0.54166667, 0.55555556])

### UPDRS_2

In [122]:
# Train the model to predict updrs_1_bool
clf2_50 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_2_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [123]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf2_50, preprocessor_X.fit_transform(X_train), y_train['updrs_2_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.56164384, 0.49315068, 0.51388889, 0.56944444, 0.56944444])

### UPDRS_3

In [124]:
# Train the model to predict updrs_1_bool
clf3_50 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_3_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [125]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf3_50, preprocessor_X.fit_transform(X_train), y_train['updrs_3_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.64383562, 0.56164384, 0.625     , 0.47222222, 0.58333333])

## 75th Percentile Training

In [126]:
target_cols = Get_Targets(clinical_data)
protein_cols = Get_ALL_Protein_Names(protein_data)

# train the model using measurements taken from months 0 and 3
X_train = df.drop(target_cols + ['visit_id', 'patient_id'], axis=1).loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = df[target_cols].loc[(df['visit_month'] == 0) | (df['visit_month'] == 3)]
y_train = y_train.drop('updrs_4', axis=1)

# drop NaN rows
# Get indices with NaN in row
inds = pd.isnull(y_train).any(1).to_numpy().nonzero()[0].tolist()
inds = y_train.index[inds].tolist()
X_train = X_train.drop(inds, axis=0)
y_train = y_train.dropna()

In [127]:
# Create columns with boolean labels that depend on whether the entry > the 25th percentile of the column
for col in y_train.columns:
    percentile_75 = y_train[col].quantile(0.75)
    y_train.loc[:, col+'_bool'] = y_train[col] > percentile_75
    
# throw out numeric columns
y_train_bool_cols = y_train.columns[3:6].tolist()
y_train = y_train[y_train_bool_cols]

### UPDRS_1

In [128]:
# Train the model to predict updrs_1_bool
clf1_75 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_1_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [129]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf1_75, preprocessor_X.fit_transform(X_train), y_train['updrs_1_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.67123288, 0.67123288, 0.66666667, 0.61111111, 0.69444444])

### UPDRS_2

In [130]:
# Train the model to predict updrs_1_bool
clf2_75 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_2_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [131]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf2_75, preprocessor_X.fit_transform(X_train), y_train['updrs_2_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.63013699, 0.61643836, 0.625     , 0.56944444, 0.55555556])

### UPDRS_3

In [132]:
# Train the model to predict updrs_1_bool
clf3_75 = LogisticRegression(fit_intercept=False, penalty='l2', solver='liblinear').fit(preprocessor_X.fit_transform(X_train),
                                                                                      y_train['updrs_3_bool'])

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


In [133]:
#Cross-Validation Method
folds = 5
skf = StratifiedKFold(folds, shuffle=True, random_state=1)
cross_val_score(clf3_75, preprocessor_X.fit_transform(X_train), y_train['updrs_3_bool'], cv=skf, scoring='accuracy')

[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s


array([0.67123288, 0.73972603, 0.59722222, 0.61111111, 0.65277778])